# Transfer Learning on the Food11 Image Dataset

This notebook demonstrates how to perform transfer learning on the **Food11 Image Dataset** using the following environment setup:

- **TensorFlow**: `2.2.0`
- **Keras**: `2.3.1`

---

## Dataset Preparation

To get started, download the dataset from [Kaggle - Food11 Image Dataset](https://www.kaggle.com/trolukovich/food11-image-dataset). Once downloaded, unzip the contents to the following directory:
`../dataset/food11-image-dataset`

The dataset contains images of food across 11 categories, which can be used for classification tasks.

---

## Model Preparation

This project leverages a pre-trained model file, `shufflenetv2_emotion_recogn.h`, provided by [opconty](https://github.com/opconty). The file contains a model pre-trained on grayscale facial expression images from the [Kaggle Emotion Recognition Challenge](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge), where the dataset includes **48x48 grayscale images**.

The process involves fine-tuning this pre-trained model for the Food11 Image Dataset to utilize transfer learning, ensuring improved accuracy and faster training.

---

### Key Steps

1. **Dataset Preprocessing**:
   - Load and preprocess the Food11 dataset.
   - Convert images to grayscale if needed.
   - Resize images to 48x48 to match the input size of the pre-trained model.

2. **Model Adjustment**:
   - Load the `shufflenetv2_emotion_recogn.h` file.
   - Replace the output layer with a fully connected layer suited for the 11-class classification problem in the Food11 dataset.
   - Fine-tune only the last few layers to avoid overfitting.
---

### Additional Notes

If you prefer training the model from scratch, I recommend using the main branch of this repository for better compatibility with the latest versions of TensorFlow and Keras.

In [ ]:
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
from datetime import datetime

In [ ]:
import sys
import os
current_dir = os.getcwd()
# Add the parent directory to sys.path
data_dir = os.path.abspath(os.path.join(current_dir, '..', 'dataset', 'food11-image-dataset'))

# Create an image data generator for image augmentation
datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)

batch_size = 16

# Create a training generator from the training folder
train_generator = datagen.flow_from_directory(
os.path.join(data_dir, 'training'),
target_size=(48, 48),
batch_size=batch_size,
class_mode='categorical',
shuffle=True,
color_mode='grayscale'
)

# Create a validation generator from the validation folder
validation_generator = datagen.flow_from_directory(
os.path.join(data_dir, 'validation'),
target_size=(48, 48),
batch_size=batch_size,
class_mode='categorical',
shuffle=False,
color_mode='grayscale'
)

# Create a evaluation generator from the evaluation folder
evaluation_generator = datagen.flow_from_directory(
os.path.join(data_dir, 'evaluation'),
target_size=(48, 48),
batch_size=batch_size,
class_mode='categorical',
shuffle=False,
color_mode='grayscale'
)

In [ ]:
# Load the base ShuffleNetV2 model
weight_path = os.path.join(current_dir, '..', 'weights', 'shufflenetv2_emotion_recogn.h5')
base_model = load_model(weight_path)

for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()

In [ ]:
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
# Add a new dense layer with 11 classes
new_output = Dense(11, activation='softmax', name='dense_11')(model.output)
# Create a new model with the modified output
model = Model(inputs=model.input, outputs=new_output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Train the model on the Food-11 dataset
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
)

In [ ]:
# Generate a timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Construct the filename with the timestamp
filename = f"shufflenetv2_{timestamp}_food_gray.h5"
model.save_weights(os.path.join(current_dir, '..', 'weights', filename))

In [ ]:
test_loss, test_accuracy = model.evaluate(evaluation_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')
print(f'Test loss: {test_loss:.4f}')

In [ ]:
# Step 5: Plot the training and validation accuracy/loss over epochs
# Accuracy Plot
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()


# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Step 6: Generate Confusion Matrix for validation set
# Get predictions for the validation set
y_pred = model.predict(validation_generator, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# True labels
y_true = validation_generator.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
cmd = ConfusionMatrixDisplay(cm, display_labels=validation_generator.class_indices.keys())
cmd.plot(cmap='Blues')
plt.title('Confusion Matrix for Validation Set')
plt.show()